#### Imports

In [1]:
import torch
import classiq

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from classiq import Model, RegisterUserInput, synthesize, show
from typing import Dict
from classiq import Model, synthesize, QReg
from classiq.builtin_functions import HardwareEfficientAnsatz
from classiq.applications.qnn import QLayer
from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection,
)

classiq.authenticate()

The current version of 'classiq' has been deprecated, and will not be supported as of 2024-01-04. Please run "pip install -U classiq" to upgrade the classiq SDK to the latest version.
f:\source\Anaconda\Lib\site-packages\classiq\_internals\authentication\token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [2]:
# constants
_NUM_QUBITS =  4
_REPS = 1
_FULLY_CONNECTED_MESH = [[0, 1], [1, 2], [2, 3], [3, 0]]
_LEARNING_RATE = 1.0

#### Classical Layer for Image Commpression:
The input MNIST images are all 28 × 28. This Classical Layer will firstly center-crop them to 24 × 24 and
then down-sample them to 4 × 4 for MNIST.

In [3]:

def classical_compression(self):
    self.center_crop = transforms.CenterCrop((24, 24))
    self.down_sample = transforms.Resize((4, 4))
    self.flatten = nn.Flatten()

In [4]:
def add_entanglement(md: Model, prefix: str, in_wire=None) -> Dict[str, QReg]:
    if in_wire is not None:
        kwargs = { "in_wires": { "IN": in_wire["OUT"] } }
    else: 
        kwargs = {}
    
    hwea_params = HardwareEfficientAnsatz(
        num_qubits=_NUM_QUBITS,
        connectivity_map=_FULLY_CONNECTED_MESH,
        reps=_REPS,
        one_qubit_gates=[],
        two_qubit_gates=["rzz", "rxx", "rzx"],
        parameter_prefix=prefix,
    )
    
    return md.HardwareEfficientAnsatz(hwea_params, **kwargs)

In [5]:
model = Model()
out1 = add_entanglement(model, "input_")
out2 = add_entanglement(model, "weight_", out1)

quantum_program = synthesize(model.get_model())

In [6]:
def execute(quantum_program: SerializedQuantumProgram, arguments: MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments)

In [7]:
# TODO: MODIFY THIS

# Post-process the result, returning a dict:
# Note: this function assumes that we only care about
#   differentiating a single state (|0>)
#   from all the rest of the states.
#   In case of a different differentiation, this function should change.
def post_process(result: SavedResult) -> torch.Tensor:
    """
    Take in a `SavedResult` with `ExecutionDetails` value type, and return the
    probability of measuring |0> which equals the amount of `|0>` measurements
    divided by the total amount of measurements.
    """
    counts: dict = result.value.counts
    # The probability of measuring |0>
    p_zero: float = counts.get("0", 0.0) / sum(counts.values())
    return torch.tensor(p_zero)

In [8]:
class Net(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute,
            post_process,
            *args,
            **kwargs
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.qlayer(x)
        return x

In [12]:
model = Net()
show(quantum_program)

Opening: https://platform.classiq.io/circuit/586246d3-7e7c-416b-bf62-ca2fcedfb997?version=0.32.1


### Data Loading

In [27]:
class CTDataset(Dataset):
    def __init__(self, filepath):
        self.x, self.y = torch.load(filepath)
        self.x = self.x / 255.
        self.y = F.one_hot(self.y, num_classes=10).to(float)
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, ix): 
        return self.x[ix], self.y[ix]

In [28]:
# load dataset
train_ds = CTDataset('MNIST_DATASET/processed/training.pt')
test_ds = CTDataset('MNIST_DATASET/processed/test.pt')
train_dl = DataLoader(train_ds, batch_size=5)

# choosing our loss function
loss_func = nn.L1Loss()

# choosing our optimizer
optimizer = optim.SGD(model.parameters(), lr=_LEARNING_RATE)

In [31]:
from typing import List

from sympy import pi

from classiq import (
    CX,
    RY,
    SWAP,
    Array,
    H,
    Input,
    Output,
    QArray,
    QBit,
    QFunc,
    QParam,
    allocate,
    control,
    create_model,
    join,
    repeat,
    split,
)

In [32]:
@QFunc
def angle_encoding(
    exe_params: QParam[List[float]], qbv: Output[QArray[QBit, "len(exe_params)"]]
) -> None:
    allocate(exe_params.len(), qbv)
    repeat(
        count=exe_params.len(),
        iteration=lambda index: RY(pi * exe_params[index], qbv[index]),
    )

In [15]:
from sympy import pi
from classiq import QFunc, QParam, QBit, H, PHASE, allocate
from classiq import create_model, synthesize, show


@QFunc
def foo(n: QParam[int], qv: QBit) -> None:
    H(qv)
    for i in range(3):  # cannot use 'n' here as it is not an int
        PHASE(theta=(i / n) * pi, target=qv)


@QFunc
def main() -> None:
    qv1 = QBit("qv1")
    allocate(1, qv1)
    foo(1, qv1)
    qv2 = QBit("qv2")
    allocate(1, qv2)
    foo(2, qv2)


model = create_model(main)
qprog = synthesize(model)
show(qprog)

Opening: https://platform.classiq.io/circuit/d8ba7220-0bef-47a7-86e3-6112e7ecdd08?version=0.32.1
